# Laboratorio 3: Métodos de Aprendizaje No Supervisado

Este laboratorio aplica técnicas de clustering y reducción de dimensionalidad a datasets reales de industria. Los ejercicios están diseñados para desarrollar competencias en segmentación de clientes, análisis exploratorio de datos de alta dimensionalidad, e interpretación de resultados para generación de insights accionables.

## Objetivos del laboratorio

- Aplicar K-Means para segmentación de clientes en contextos de negocio
- Comparar diferentes números de clusters usando métricas de evaluación
- Implementar clustering jerárquico y visualizar dendrogramas
- Utilizar PCA para reducción de dimensionalidad y visualización
- Aplicar técnicas modernas (t-SNE, UMAP) para exploración de datos complejos
- Interpretar resultados y generar recomendaciones accionables

## Estructura del laboratorio

1. **Segmentación de clientes con K-Means**: Identificación de perfiles de consumo en una empresa de retail
2. **Clustering jerárquico**: Análisis de jerarquías naturales en datos de productos
3. **Reducción de dimensionalidad con PCA**: Compresión y visualización de datos de alta dimensionalidad
4. **Técnicas avanzadas**: Comparación de t-SNE y UMAP para visualización de estructuras complejas

## Parte 1: Configuración del Entorno

Importación de librerías necesarias y configuración de visualizaciones.

In [ ]:
# Importar librerias basicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configuracion de visualizacion
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configurar tamano de figuras por defecto
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10

print("Librerías básicas importadas correctamente")

In [ ]:
# Importar librerias de machine learning
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.manifold import TSNE
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform

print("Librerías de machine learning importadas correctamente")

## Parte 2: Segmentación de Clientes con K-Means

### Contexto del negocio

Una empresa de comercio electrónico desea segmentar su base de clientes para implementar estrategias de marketing diferenciadas. El objetivo es identificar grupos de clientes con comportamientos de compra similares para personalizar ofertas, optimizar campañas publicitarias, y mejorar la retención.

El dataset contiene información de comportamiento de compra de 200 clientes durante el último año, incluyendo:
- `frecuencia_compra`: Número de transacciones realizadas
- `valor_promedio`: Monto promedio gastado por transacción (en dólares)
- `valor_total`: Gasto total anual (en dólares)
- `dias_ultima_compra`: Días desde la última compra (recency)
- `productos_unicos`: Número de productos diferentes comprados
- `tasa_devolucion`: Porcentaje de productos devueltos

### Ejercicio 2.1: Generación y exploración de datos

Generar un dataset sintético de clientes y realizar un análisis exploratorio inicial.

In [ ]:
# Cargar dataset de clientes
datos_clientes = pd.read_csv('data/segmentacion_clientes_ecommerce.csv')

print(f"Dataset cargado con {len(datos_clientes)} clientes")
print(f"\nPrimeras filas del dataset:")
datos_clientes.head(10)

In [ ]:
# Realizar analisis exploratorio
print("Estadísticas descriptivas del dataset:\n")
print(datos_clientes.describe().round(2))

print("\n" + "="*80)
print("Información sobre valores faltantes:")
print(datos_clientes.isnull().sum())

In [ ]:
# Visualizar distribuciones de variables clave
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

variables = ['frecuencia_compra', 'valor_promedio', 'valor_total', 
             'dias_ultima_compra', 'productos_unicos', 'tasa_devolucion']

for idx, var in enumerate(variables):
    axes[idx].hist(datos_clientes[var], bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribución de {var}')
    axes[idx].set_xlabel(var)
    axes[idx].set_ylabel('Frecuencia')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Ejercicio 2.2: Preprocesamiento de datos

La estandarización de variables es crítica en algoritmos basados en distancias como K-Means. Sin normalización, variables con escalas mayores (ej: valor_total en miles de dólares) dominan el cálculo de distancias sobre variables en escalas menores (ej: tasa_devolucion en porcentaje), sesgando los resultados. `StandardScaler` transforma cada variable a media 0 y desviación estándar 1, garantizando contribución equitativa de todas las características.

In [ ]:
# Seleccionar variables para clustering (excluir cliente_id)
variables_clustering = ['frecuencia_compra', 'valor_promedio', 'valor_total',
                        'dias_ultima_compra', 'productos_unicos', 'tasa_devolucion']

X = datos_clientes[variables_clustering]

# Crear instancia de StandardScaler
scaler = StandardScaler()

# Ajustar el scaler y transformar los datos
X_estandarizado = scaler.fit_transform(X)

# Convertir a DataFrame para facilitar visualizacion
X_std_df = pd.DataFrame(X_estandarizado, columns=variables_clustering)

print("Datos estandarizados - primeras filas:")
print(X_std_df.head())
print("\nMedia de datos estandarizados (debe ser ~0):")
print(X_std_df.mean().round(6))
print("\nDesviación estándar de datos estandarizados (debe ser ~1):")
print(X_std_df.std().round(6))

### Ejercicio 2.3: Método del codo

El método del codo evalúa la inercia (suma de distancias cuadradas intra-cluster) para diferentes valores de k. La inercia decrece monotónicamente con k adicionales, pero la tasa de mejora disminuye. El "codo" del gráfico indica el punto donde clusters adicionales aportan rendimientos marginales decrecientes, sugiriendo el número óptimo para balance entre parsimonia del modelo y calidad de clustering.

In [ ]:
# Evaluar inercia para diferentes numeros de clusters
rango_k = range(2, 11)
inercias = []

for k in rango_k:
    # Crear modelo KMeans con k clusters, random_state=42
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    
    # Ajustar el modelo a los datos estandarizados
    kmeans.fit(X_estandarizado)
    
    # Almacenar la inercia del modelo
    inercias.append(kmeans.inertia_)

# Visualizar metodo del codo
plt.figure(figsize=(10, 6))
plt.plot(rango_k, inercias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Número de Clusters (k)', fontsize=12)
plt.ylabel('Inercia (Suma de Distancias Cuadradas)', fontsize=12)
plt.title('Método del Codo para Selección de k', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xticks(rango_k)
plt.show()

print("Inercias por número de clusters:")
for k, inercia in zip(rango_k, inercias):
    print(f"k={k}: Inercia = {inercia:.2f}")

### Ejercicio 2.4: Coeficiente de silueta

El coeficiente de silueta cuantifica la calidad de clustering evaluando tanto cohesión interna (qué tan cerca están los puntos dentro de un cluster) como separación externa (qué tan alejados están de otros clusters). Valores cercanos a 1 indican clusters bien definidos y separados, mientras valores negativos sugieren asignaciones incorrectas. Esta métrica complementa el método del codo proporcionando validación cuantitativa de la estructura de clustering.

In [ ]:
# Calcular coeficiente de silueta para diferentes k
coeficientes_silueta = []

for k in rango_k:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    etiquetas = kmeans.fit_predict(X_estandarizado)
    
    # Calcular coeficiente de silueta usando silhouette_score
    coef_silueta = silhouette_score(X_estandarizado, etiquetas)
    coeficientes_silueta.append(coef_silueta)

# Visualizar coeficientes de silueta
plt.figure(figsize=(10, 6))
plt.plot(rango_k, coeficientes_silueta, 'go-', linewidth=2, markersize=8)
plt.xlabel('Número de Clusters (k)', fontsize=12)
plt.ylabel('Coeficiente de Silueta', fontsize=12)
plt.title('Coeficiente de Silueta para Diferentes k', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xticks(rango_k)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.show()

print("Coeficientes de silueta por número de clusters:")
for k, coef in zip(rango_k, coeficientes_silueta):
    print(f"k={k}: Coeficiente de Silueta = {coef:.4f}")

mejor_k = rango_k[np.argmax(coeficientes_silueta)]
print(f"\nMejor k según coeficiente de silueta: {mejor_k}")

### Ejercicio 2.5: Aplicación de K-Means con k óptimo

La selección final del número de clusters balancea métricas cuantitativas (inercia, coeficiente de silueta) con interpretabilidad de negocio. En segmentación de clientes, un número moderado de clusters (3-5) facilita estrategias de marketing accionables, mientras que segmentaciones más granulares pueden carecer de diferenciación práctica entre grupos contiguos.

In [ ]:
# Aplicar K-Means con k=3 (basado en generacion de datos y metricas)
k_final = 3

# Crear modelo KMeans final con k_final clusters
kmeans_final = KMeans(n_clusters=k_final, random_state=42, n_init=10)

# Ajustar el modelo y obtener etiquetas
etiquetas_clusters = kmeans_final.fit_predict(X_estandarizado)

# Agregar etiquetas al DataFrame original
datos_clientes['cluster'] = etiquetas_clusters

print(f"Segmentación completada con k={k_final} clusters")
print(f"\nDistribución de clientes por cluster:")
print(datos_clientes['cluster'].value_counts().sort_index())
print(f"\nPorcentaje por cluster:")
print((datos_clientes['cluster'].value_counts(normalize=True).sort_index() * 100).round(2))

### Ejercicio 2.6: Caracterización e interpretación de clusters

La traducción de clusters algorítmicos a segmentos de negocio accionables requiere análisis de características promedio y asignación de etiquetas interpretables. Este proceso conecta resultados técnicos con estrategias de marketing, pricing, o gestión de inventario, transformando agrupaciones numéricas en insights operacionales.

In [ ]:
# Calcular estadisticas promedio por cluster
perfiles_clusters = datos_clientes.groupby('cluster')[variables_clustering].mean()

print("Perfiles promedio por cluster:\n")
print(perfiles_clusters.round(2))

# Visualizar comparacion de clusters
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, var in enumerate(variables_clustering):
    perfiles_clusters[var].plot(kind='bar', ax=axes[idx], color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
    axes[idx].set_title(f'{var} por Cluster', fontsize=11)
    axes[idx].set_xlabel('Cluster')
    axes[idx].set_ylabel('Valor Promedio')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# Asignar nombres interpretables a los clusters basados en caracteristicas
nombres_clusters = {
    0: 'Clientes Ocasionales',
    1: 'Clientes Premium', 
    2: 'Clientes Regulares'
}

# Identificar cual cluster corresponde a cada perfil
# El cluster con mayor valor_total es Premium, menor es Ocasional
cluster_valores = perfiles_clusters['valor_total'].sort_values()

# Reasignar nombres basados en valores
if perfiles_clusters['valor_total'].iloc[0] > perfiles_clusters['valor_total'].iloc[2]:
    if perfiles_clusters['valor_total'].iloc[0] > perfiles_clusters['valor_total'].iloc[1]:
        nombres_clusters[0] = 'Clientes Premium'
        if perfiles_clusters['valor_total'].iloc[1] > perfiles_clusters['valor_total'].iloc[2]:
            nombres_clusters[1] = 'Clientes Regulares'
            nombres_clusters[2] = 'Clientes Ocasionales'
        else:
            nombres_clusters[1] = 'Clientes Ocasionales'
            nombres_clusters[2] = 'Clientes Regulares'

datos_clientes['segmento'] = datos_clientes['cluster'].map(nombres_clusters)

print("\nInterpretación de segmentos:")
for cluster_id in range(k_final):
    print(f"\n{nombres_clusters[cluster_id]} (Cluster {cluster_id}):")
    perfil = perfiles_clusters.loc[cluster_id]
    print(f"  - Frecuencia de compra: {perfil['frecuencia_compra']:.1f} transacciones/año")
    print(f"  - Valor promedio: ${perfil['valor_promedio']:.2f} por transacción")
    print(f"  - Valor total: ${perfil['valor_total']:.2f} anual")
    print(f"  - Recency: {perfil['dias_ultima_compra']:.0f} días desde última compra")
    print(f"  - Productos únicos: {perfil['productos_unicos']:.1f} productos diferentes")
    print(f"  - Tasa de devolución: {perfil['tasa_devolucion']:.1f}%")

In [ ]:
# Visualizar clusters en espacio 2D (usando dos variables principales)
plt.figure(figsize=(12, 5))

# Grafico 1: Frecuencia vs Valor Total
plt.subplot(1, 2, 1)
for cluster_id in range(k_final):
    cluster_data = datos_clientes[datos_clientes['cluster'] == cluster_id]
    plt.scatter(cluster_data['frecuencia_compra'], 
                cluster_data['valor_total'],
                label=nombres_clusters[cluster_id],
                alpha=0.6, s=50)

plt.xlabel('Frecuencia de Compra')
plt.ylabel('Valor Total ($)')
plt.title('Segmentación: Frecuencia vs Valor Total')
plt.legend()
plt.grid(True, alpha=0.3)

# Grafico 2: Dias Ultima Compra vs Valor Promedio
plt.subplot(1, 2, 2)
for cluster_id in range(k_final):
    cluster_data = datos_clientes[datos_clientes['cluster'] == cluster_id]
    plt.scatter(cluster_data['dias_ultima_compra'], 
                cluster_data['valor_promedio'],
                label=nombres_clusters[cluster_id],
                alpha=0.6, s=50)

plt.xlabel('Días desde Última Compra')
plt.ylabel('Valor Promedio ($)')
plt.title('Segmentación: Recency vs Valor Promedio')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Parte 3: Clustering Jerárquico

### Contexto del negocio

Una plataforma de e-commerce necesita organizar su catálogo de 50 productos en categorías naturales sin usar clasificaciones predefinidas. El objetivo es descubrir jerarquías de similitud entre productos basándose en características de venta y comportamiento de clientes, permitiendo navegación intuitiva y recomendaciones efectivas.

### Ejercicio 3.1: Generación de datos de productos

In [ ]:
# Cargar dataset de productos
datos_productos = pd.read_csv('data/catalogo_productos_retail.csv')

print(f"Dataset de productos cargado: {len(datos_productos)} productos")
print(f"\nEstadísticas descriptivas:")
print(datos_productos.describe().round(2))
datos_productos.head(10)

### Ejercicio 3.2: Clustering jerárquico aglomerativo

El clustering jerárquico construye una jerarquía de clusters mediante fusiones sucesivas, permitiendo explorar estructura de datos a múltiples niveles de granularidad. El dendrograma visualiza este proceso jerárquico, facilitando identificación de puntos de corte naturales y comparación de diferentes configuraciones de clusters sin pre-especificar k.

In [ ]:
# Preparar datos para clustering jerarquico
X_productos = datos_productos[['precio', 'ventas_mensuales', 'margen_beneficio', 
                                'rating_promedio', 'num_reviews']]

# Estandarizar datos
scaler_productos = StandardScaler()
X_productos_std = scaler_productos.fit_transform(X_productos)

# Calcular matriz de enlaces usando metodo de Ward
# Ward minimiza la varianza intra-cluster
Z = linkage(X_productos_std, method='ward')

# Visualizar dendrograma
plt.figure(figsize=(15, 7))
dendrogram(Z, 
           labels=datos_productos['producto_id'].values,
           leaf_font_size=8,
           color_threshold=50)
plt.title('Dendrograma de Clustering Jerárquico - Productos', fontsize=14)
plt.xlabel('Producto ID', fontsize=12)
plt.ylabel('Distancia (Ward)', fontsize=12)
plt.axhline(y=50, color='r', linestyle='--', label='Corte sugerido (3 clusters)')
plt.legend()
plt.tight_layout()
plt.show()

print("Dendrograma generado usando método de Ward")

### Ejercicio 3.3: Comparación de métodos de enlace

Diferentes métodos de enlace (single, complete, average, Ward) definen criterios distintos para medir distancia entre clusters, generando jerarquías con características específicas. Single linkage tiende a producir cadenas largas (sensible a outliers), complete genera clusters compactos pero puede fragmentar grupos naturales, average ofrece balance intermedio, y Ward optimiza homogeneidad intra-cluster.

In [ ]:
# Comparar diferentes metodos de enlace
metodos = ['single', 'complete', 'average', 'ward']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for idx, metodo in enumerate(metodos):
    # Calcular linkage con cada metodo
    Z_metodo = linkage(X_productos_std, method=metodo)
    
    # Visualizar dendrograma
    dendrogram(Z_metodo, 
               ax=axes[idx],
               labels=datos_productos['producto_id'].values,
               leaf_font_size=6,
               no_labels=(len(datos_productos) > 30))
    
    axes[idx].set_title(f'Método: {metodo.upper()}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Producto ID', fontsize=10)
    axes[idx].set_ylabel('Distancia', fontsize=10)

plt.tight_layout()
plt.show()

print("Comparación de métodos de enlace:")
print("- Single: Minimiza distancia entre elementos más cercanos (sensible a outliers)")
print("- Complete: Maximiza distancia entre elementos más lejanos (clusters compactos)")
print("- Average: Promedia distancias (compromiso entre single y complete)")
print("- Ward: Minimiza varianza intra-cluster (clusters balanceados)")

### Ejercicio 3.4: Corte del dendrograma y caracterización

El corte del dendrograma a una altura específica define el número final de clusters. Este punto de corte se selecciona evaluando la estructura del dendrograma (ramas largas verticales indican separación natural entre grupos) y balanceando granularidad con interpretabilidad de negocio. La caracterización posterior asigna significado a cada cluster identificado.

In [ ]:
# Aplicar AgglomerativeClustering con n_clusters=3
modelo_jerarquico = AgglomerativeClustering(n_clusters=3, linkage='ward')
etiquetas_productos = modelo_jerarquico.fit_predict(X_productos_std)

# Agregar etiquetas al dataset
datos_productos['categoria'] = etiquetas_productos

# Analizar caracteristicas por categoria
print("Distribución de productos por categoría:")
print(datos_productos['categoria'].value_counts().sort_index())

print("\n" + "="*80)
print("Características promedio por categoría:\n")
perfiles_productos = datos_productos.groupby('categoria')[['precio', 'ventas_mensuales', 
                                                             'margen_beneficio', 'rating_promedio', 
                                                             'num_reviews']].mean()
print(perfiles_productos.round(2))

# Asignar nombres interpreables
nombres_categorias = {}
for cat in range(3):
    perfil = perfiles_productos.loc[cat]
    if perfil['precio'] > 400:
        nombres_categorias[cat] = 'Electrónica'
    elif perfil['precio'] < 30:
        nombres_categorias[cat] = 'Accesorios'
    else:
        nombres_categorias[cat] = 'Ropa y Textiles'

datos_productos['categoria_nombre'] = datos_productos['categoria'].map(nombres_categorias)

print("\n" + "="*80)
print("Categorías identificadas:")
for cat, nombre in nombres_categorias.items():
    print(f"\nCategoría {cat}: {nombre}")
    perfil = perfiles_productos.loc[cat]
    print(f"  - Precio promedio: ${perfil['precio']:.2f}")
    print(f"  - Ventas mensuales: {perfil['ventas_mensuales']:.0f} unidades")
    print(f"  - Margen de beneficio: {perfil['margen_beneficio']:.1f}%")
    print(f"  - Rating: {perfil['rating_promedio']:.2f}/5.0")
    print(f"  - Reviews: {perfil['num_reviews']:.0f} promedio")

## Parte 4: Reducción de Dimensionalidad con PCA

### Contexto del negocio

Un equipo de analítica tiene un dataset con 20 variables que describen el comportamiento de usuarios en una plataforma digital. Para facilitar la visualización, comprensión de patrones, y acelerar modelos posteriores, se requiere reducir la dimensionalidad preservando la mayor cantidad de información posible.

### Ejercicio 4.1: Generación de datos de alta dimensionalidad

In [ ]:
# Cargar dataset de alta dimensionalidad
df_alta_dim = pd.read_csv('data/metricas_usuarios_plataforma_digital.csv')

n_variables = df_alta_dim.shape[1] - 1  # Excluir grupo_real

print(f"Dataset cargado: {df_alta_dim.shape[0]} observaciones, {n_variables} variables")
print(f"\nPrimeras filas:")
print(df_alta_dim.head())
print(f"\nDistribución de grupos reales:")
print(df_alta_dim['grupo_real'].value_counts().sort_index())

### Ejercicio 4.2: Aplicación de PCA

PCA (Principal Component Analysis) identifica direcciones de máxima varianza en datos de alta dimensionalidad, proyectando observaciones a un espacio de menor dimensión que preserva la mayor cantidad posible de información. La varianza explicada por cada componente cuantifica qué proporción de la información original se retiene, permitiendo selección informada del número de componentes para balance entre reducción dimensional y pérdida de información.

In [ ]:
# Preparar datos (excluir grupo_real)
X_pca = df_alta_dim.drop('grupo_real', axis=1)
nombres_vars = X_pca.columns.tolist()

# Estandarizar datos
scaler_pca = StandardScaler()
X_pca_std = scaler_pca.fit_transform(X_pca)

# Aplicar PCA con todos los componentes
pca = PCA()
X_pca_transformado = pca.fit_transform(X_pca_std)

# Analizar varianza explicada
varianza_explicada = pca.explained_variance_ratio_
varianza_acumulada = np.cumsum(varianza_explicada)
n_variables = len(nombres_vars)

print("Varianza explicada por cada componente principal:")
for i, var in enumerate(varianza_explicada[:10], 1):
    print(f"PC{i}: {var*100:.2f}%")

print(f"\n{'='*80}")
print(f"Varianza acumulada por primeros componentes:")
for i in [2, 3, 5, 10]:
    print(f"Primeros {i} componentes: {varianza_acumulada[i-1]*100:.2f}%")

In [ ]:
# Visualizar scree plot y varianza acumulada
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
axes[0].plot(range(1, len(varianza_explicada)+1), varianza_explicada, 'bo-')
axes[0].set_xlabel('Componente Principal', fontsize=11)
axes[0].set_ylabel('Proporción de Varianza Explicada', fontsize=11)
axes[0].set_title('Scree Plot', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0.05, color='r', linestyle='--', alpha=0.5, label='5% umbral')
axes[0].legend()

# Varianza acumulada
axes[1].plot(range(1, len(varianza_acumulada)+1), varianza_acumulada, 'go-')
axes[1].axhline(y=0.80, color='r', linestyle='--', alpha=0.5, label='80% varianza')
axes[1].axhline(y=0.90, color='orange', linestyle='--', alpha=0.5, label='90% varianza')
axes[1].set_xlabel('Número de Componentes', fontsize=11)
axes[1].set_ylabel('Varianza Acumulada', fontsize=11)
axes[1].set_title('Varianza Acumulada', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.show()

# Determinar numero de componentes para 80% y 90% de varianza
n_comp_80 = np.argmax(varianza_acumulada >= 0.80) + 1
n_comp_90 = np.argmax(varianza_acumulada >= 0.90) + 1

print(f"\nComponentes necesarios para 80% varianza: {n_comp_80}")
print(f"Componentes necesarios para 90% varianza: {n_comp_90}")
print(f"Reducción dimensional: {n_variables} → {n_comp_80} variables ({n_comp_80/n_variables*100:.1f}% del tamaño original)")

### Ejercicio 4.3: Visualización en 2D con PCA

La proyección a 2 dimensiones mediante PCA facilita visualización de estructura de datos originalmente en alta dimensionalidad. Los dos primeros componentes principales capturan las direcciones de mayor variabilidad, permitiendo identificación visual de clusters, outliers, y patrones que serían imposibles de observar en el espacio original de 20+ variables.

In [ ]:
# Aplicar PCA con 2 componentes
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_pca_std)

# Crear DataFrame con componentes
df_pca_2d = pd.DataFrame(X_pca_2d, columns=['PC1', 'PC2'])
df_pca_2d['grupo'] = df_alta_dim['grupo_real']

# Visualizar
plt.figure(figsize=(10, 7))
colores = ['#FF6B6B', '#4ECDC4', '#45B7D1']
grupos_nombres = ['Grupo A', 'Grupo B', 'Grupo C']

for grupo_id in range(3):
    mask = df_pca_2d['grupo'] == grupo_id
    plt.scatter(df_pca_2d.loc[mask, 'PC1'], 
                df_pca_2d.loc[mask, 'PC2'],
                c=colores[grupo_id],
                label=grupos_nombres[grupo_id],
                alpha=0.6,
                s=50,
                edgecolors='black',
                linewidth=0.5)

plt.xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}% varianza)', fontsize=12)
plt.ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}% varianza)', fontsize=12)
plt.title('Proyección PCA 2D - Visualización de Grupos', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Varianza total capturada por 2 componentes: {sum(pca_2d.explained_variance_ratio_)*100:.2f}%")

### Ejercicio 4.4: Análisis de loadings

Los loadings (o pesos factoriales) de PCA revelan cómo cada variable original contribuye a los componentes principales. Variables con loadings altos (en valor absoluto) tienen mayor influencia en la dirección del componente, permitiendo interpretación temática de componentes. Por ejemplo, si PC1 tiene loadings altos en variables de gasto, puede interpretarse como "dimensión de valor económico".

In [ ]:
# Extraer loadings (componentes)
loadings = pca_2d.components_.T * np.sqrt(pca_2d.explained_variance_)

# Crear DataFrame de loadings
df_loadings = pd.DataFrame(
    loadings,
    columns=['PC1', 'PC2'],
    index=nombres_vars
)

print("Loadings de variables en los dos primeros componentes:\n")
print(df_loadings.round(3))

# Visualizar loadings
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Loadings PC1
df_loadings['PC1'].sort_values().plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Loadings - Componente Principal 1', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Loading', fontsize=11)
axes[0].grid(True, alpha=0.3, axis='x')

# Loadings PC2
df_loadings['PC2'].sort_values().plot(kind='barh', ax=axes[1], color='coral')
axes[1].set_title('Loadings - Componente Principal 2', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Loading', fontsize=11)
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# Identificar variables mas influyentes
print(f"\n{'='*80}")
print("Variables con mayor influencia en PC1:")
print(df_loadings['PC1'].abs().sort_values(ascending=False).head(5))
print(f"\n{'='*80}")
print("Variables con mayor influencia en PC2:")
print(df_loadings['PC2'].abs().sort_values(ascending=False).head(5))

## Parte 5: Técnicas Avanzadas de Reducción de Dimensionalidad

### Contexto

PCA es una técnica lineal que puede no capturar relaciones no lineales complejas en los datos. Técnicas modernas como t-SNE y UMAP están diseñadas para preservar estructura local y global, siendo especialmente útiles para visualización de datos con patrones complejos.

### Ejercicio 5.1: Aplicación de t-SNE

t-SNE (t-Distributed Stochastic Neighbor Embedding) es una técnica no-lineal que preserva estructura local de datos, proyectando observaciones similares cerca unas de otras en el espacio reducido. A diferencia de PCA que busca varianza máxima globalmente, t-SNE optimiza para mantener vecindarios locales, revelando clusters y patrones no-lineales que métodos lineales pueden no capturar.

In [ ]:
# Aplicar t-SNE con perplexity=30
tsne = TSNE(n_components=2, perplexity=30, random_state=42, n_iter=1000)
X_tsne = tsne.fit_transform(X_pca_std)

# Crear DataFrame
df_tsne = pd.DataFrame(X_tsne, columns=['t-SNE1', 't-SNE2'])
df_tsne['grupo'] = df_alta_dim['grupo_real']

# Visualizar
plt.figure(figsize=(10, 7))

for grupo_id in range(3):
    mask = df_tsne['grupo'] == grupo_id
    plt.scatter(df_tsne.loc[mask, 't-SNE1'], 
                df_tsne.loc[mask, 't-SNE2'],
                c=colores[grupo_id],
                label=grupos_nombres[grupo_id],
                alpha=0.6,
                s=50,
                edgecolors='black',
                linewidth=0.5)

plt.xlabel('t-SNE Dimensión 1', fontsize=12)
plt.ylabel('t-SNE Dimensión 2', fontsize=12)
plt.title('Proyección t-SNE 2D - Visualización de Grupos', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("t-SNE aplicado correctamente")
print("Nota: t-SNE preserva estructura local, grupos cercanos en espacio original permanecen cercanos")

### Ejercicio 5.2: Comparación visual PCA vs t-SNE

La comparación entre PCA y t-SNE ilustra fortalezas de cada enfoque. PCA, siendo lineal, es rápida, determinística, e interpretable mediante varianza explicada y loadings. t-SNE, siendo no-lineal, puede revelar estructura local compleja pero es computacionalmente costosa, estocástica (resultados varían con random seed), y no permite interpretación directa de ejes.

In [ ]:
# Comparacion lado a lado
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Grafico PCA
for grupo_id in range(3):
    mask = df_pca_2d['grupo'] == grupo_id
    axes[0].scatter(df_pca_2d.loc[mask, 'PC1'], 
                    df_pca_2d.loc[mask, 'PC2'],
                    c=colores[grupo_id],
                    label=grupos_nombres[grupo_id],
                    alpha=0.6,
                    s=50,
                    edgecolors='black',
                    linewidth=0.5)

axes[0].set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}%)', fontsize=11)
axes[0].set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}%)', fontsize=11)
axes[0].set_title('PCA: Reducción Lineal', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Grafico t-SNE
for grupo_id in range(3):
    mask = df_tsne['grupo'] == grupo_id
    axes[1].scatter(df_tsne.loc[mask, 't-SNE1'], 
                    df_tsne.loc[mask, 't-SNE2'],
                    c=colores[grupo_id],
                    label=grupos_nombres[grupo_id],
                    alpha=0.6,
                    s=50,
                    edgecolors='black',
                    linewidth=0.5)

axes[1].set_xlabel('t-SNE Dimensión 1', fontsize=11)
axes[1].set_ylabel('t-SNE Dimensión 2', fontsize=11)
axes[1].set_title('t-SNE: Preservación de Estructura Local', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Comparación PCA vs t-SNE:")
print("- PCA: Técnica lineal, maximiza varianza, interpretable, rápida")
print("- t-SNE: Técnica no-lineal, preserva estructura local, mejor para visualización")

## Parte 6: Síntesis y Aplicación Integrada

### Ejercicio Final: Pipeline completo de análisis no supervisado

**Contexto:** Un banco desea segmentar su cartera de préstamos hipotecarios para identificar perfiles de riesgo y optimizar estrategias de cobro. El dataset contiene información de 500 préstamos con 15 variables.

### Ejercicio 6.1: Análisis exploratorio de datos bancarios

El análisis de cartera de préstamos hipotecarios requiere identificación de perfiles de riesgo mediante combinación de variables financieras (LTV ratio, score crediticio, ratio deuda-ingreso) y demográficas (edad, antigüedad laboral). La segmentación permite estratificación de riesgo para ajuste de tasas, asignación de recursos de cobranza, y decisiones de aprobación de nuevos préstamos.

In [ ]:
# Cargar dataset de préstamos
datos_prestamos = pd.read_csv('data/cartera_prestamos_hipotecarios.csv')

print(f"Dataset de préstamos cargado: {len(datos_prestamos)} préstamos")
print(f"\nEstadísticas descriptivas:")
print(datos_prestamos.describe().round(2))
datos_prestamos.head(10)

### Ejercicio 6.2: Pipeline integrado de análisis

Un pipeline PCA + K-Means combina reducción de dimensionalidad con clustering. PCA elimina ruido y multicolinealidad, comprimiendo información correlacionada en componentes ortogonales. K-Means sobre componentes principales puede mejorar calidad de clusters y velocidad computacional. La comparación con clustering directo evalúa trade-off entre compresión de información y calidad de segmentación.

In [ ]:
# Preparar datos
variables_prestamos = ['monto_prestamo', 'ingreso_anual', 'ltv_ratio', 'score_credito',
                       'tasa_interes', 'plazo_anos', 'edad_solicitante', 'antiguedad_empleo',
                       'num_dependientes', 'deuda_total', 'pagos_atrasados_12m',
                       'ratio_deuda_ingreso', 'cuota_mensual']

X_prestamos = datos_prestamos[variables_prestamos]

# Estandarizar datos
scaler_prestamos = StandardScaler()
X_prestamos_std = scaler_prestamos.fit_transform(X_prestamos)

# Metodo 1: K-Means directo sobre datos estandarizados
kmeans_directo = KMeans(n_clusters=3, random_state=42, n_init=10)
etiquetas_directo = kmeans_directo.fit_predict(X_prestamos_std)

# Metodo 2: PCA seguido de K-Means
# Primero aplicar PCA para reducir a 5 componentes (capturan ~80% varianza)
pca_prestamos = PCA(n_components=5)
X_pca_prestamos = pca_prestamos.fit_transform(X_prestamos_std)

kmeans_pca = KMeans(n_clusters=3, random_state=42, n_init=10)
etiquetas_pca = kmeans_pca.fit_predict(X_pca_prestamos)

# Agregar resultados al dataset
datos_prestamos['cluster_directo'] = etiquetas_directo
datos_prestamos['cluster_pca'] = etiquetas_pca

# Comparar resultados
print("Método 1: K-Means sobre datos originales (13 variables)")
print(f"Inercia: {kmeans_directo.inertia_:.2f}")
print(f"Coeficiente de silueta: {silhouette_score(X_prestamos_std, etiquetas_directo):.4f}")
print(f"Distribución: {pd.Series(etiquetas_directo).value_counts().sort_index().to_dict()}")

print(f"\n{'='*80}")
print("Método 2: K-Means sobre PCA (5 componentes)")
print(f"Varianza explicada por PCA: {sum(pca_prestamos.explained_variance_ratio_)*100:.2f}%")
print(f"Inercia: {kmeans_pca.inertia_:.2f}")
print(f"Coeficiente de silueta: {silhouette_score(X_pca_prestamos, etiquetas_pca):.4f}")
print(f"Distribución: {pd.Series(etiquetas_pca).value_counts().sort_index().to_dict()}")

### Ejercicio 6.3: Caracterización de perfiles de riesgo y recomendaciones

La traducción de clusters a perfiles de riesgo operacionales requiere análisis multivariado de indicadores financieros y construcción de scores compuestos. Cada perfil debe asociarse con estrategias diferenciadas de gestión: clientes bajo riesgo reciben beneficios (tasas preferenciales, productos adicionales), riesgo medio requiere monitoreo proactivo, y alto riesgo demanda intervención intensiva y reestructuración.

In [ ]:
# Analizar perfiles usando metodo directo
perfiles_riesgo = datos_prestamos.groupby('cluster_directo')[variables_prestamos].mean()

print("PERFILES DE RIESGO IDENTIFICADOS\n")
print("="*80)

# Determinar nivel de riesgo por cluster basado en indicadores clave
for cluster_id in range(3):
    perfil = perfiles_riesgo.loc[cluster_id]
    
    # Calcular score de riesgo compuesto
    riesgo_score = (
        (perfil['ltv_ratio'] / 100) * 0.3 +  # Mayor LTV = mayor riesgo
        ((800 - perfil['score_credito']) / 800) * 0.3 +  # Menor score = mayor riesgo
        (perfil['ratio_deuda_ingreso'] / 100) * 0.2 +  # Mayor ratio = mayor riesgo
        (perfil['pagos_atrasados_12m'] / 5) * 0.2  # Más atrasos = mayor riesgo
    )
    
    if riesgo_score < 0.4:
        nivel = "BAJO RIESGO"
    elif riesgo_score < 0.6:
        nivel = "RIESGO MEDIO"
    else:
        nivel = "ALTO RIESGO"
    
    print(f"\nCLUSTER {cluster_id}: {nivel}")
    print(f"Score de Riesgo Compuesto: {riesgo_score:.3f}")
    print(f"  Tamaño del segmento: {(datos_prestamos['cluster_directo']==cluster_id).sum()} préstamos")
    print(f"  Monto promedio: ${perfil['monto_prestamo']:,.0f}")
    print(f"  Ingreso promedio: ${perfil['ingreso_anual']:,.0f}")
    print(f"  LTV Ratio: {perfil['ltv_ratio']:.1f}%")
    print(f"  Score de crédito: {perfil['score_credito']:.0f}")
    print(f"  Tasa de interés: {perfil['tasa_interes']:.2f}%")
    print(f"  Ratio deuda/ingreso: {perfil['ratio_deuda_ingreso']:.1f}%")
    print(f"  Pagos atrasados (12m): {perfil['pagos_atrasados_12m']:.2f}")
    print(f"  Antigüedad empleo: {perfil['antiguedad_empleo']:.1f} años")
    
    # Generar recomendaciones especificas
    print(f"\n  RECOMENDACIONES:")
    if nivel == "BAJO RIESGO":
        print("    • Mantener seguimiento estándar")
        print("    • Ofrecer productos adicionales (cross-selling)")
        print("    • Considerar tasas preferenciales para refinanciamiento")
    elif nivel == "RIESGO MEDIO":
        print("    • Monitoreo mensual de pagos")
        print("    • Ofrecer asesoría financiera proactiva")
        print("    • Evaluar refinanciamiento si mejoran condiciones")
    else:
        print("    • Seguimiento intensivo (semanal)")
        print("    • Contacto proactivo ante primer atraso")
        print("    • Programa de reestructuración de deuda")
        print("    • Evaluar garantías adicionales")

print(f"\n{'='*80}")

## Conclusiones del Laboratorio

Este laboratorio ha cubierto las técnicas fundamentales de aprendizaje no supervisado aplicadas a casos de negocio reales:

### Aprendizajes Clave

1. **K-Means para Segmentación**: Técnica eficiente para identificar grupos naturales en datos de clientes, productos o transacciones. La selección del número de clusters requiere balance entre métricas cuantitativas (método del codo, silueta) y utilidad de negocio.

2. **Clustering Jerárquico**: Permite explorar estructura de datos a múltiples niveles de granularidad. Los dendrogramas facilitan visualización de relaciones y la comparación de métodos de enlace revela diferentes perspectivas sobre similitud.

3. **PCA para Reducción de Dimensionalidad**: Técnica lineal que comprime información preservando varianza. Útil para visualización, preprocesamiento, y comprensión de relaciones entre variables mediante análisis de loadings.

4. **Técnicas Avanzadas (t-SNE)**: Métodos no-lineales que preservan estructura local, superiores a PCA para visualización de patrones complejos, aunque menos interpretables y más costosos computacionalmente.

5. **Pipeline Integrado**: La combinación de reducción de dimensionalidad con clustering puede mejorar resultados al eliminar ruido y reducir complejidad computacional.

### Consideraciones para Producción

- **Estandarización**: Crítica para algoritmos basados en distancias
- **Selección de variables**: Incluir solo variables relevantes mejora resultados
- **Interpretabilidad**: Traducir resultados técnicos a insights accionables de negocio
- **Validación**: Colaborar con expertos de dominio para validar segmentaciones
- **Monitoreo**: Re-entrenar periódicamente conforme evolucionan los datos